In [ ]:
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)


# Klassifikation der MNIST Ziffern

Wir laden uns die 60.000 Trainings- und 10.000 Test-Bilder im Format 28x28.

**Beachte:** Die Daten werden einmalig heruntergeladen und lokal persistiert - das erste Mal kann abhängig von Bandbreite etwas länger dauern.

In [ ]:
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_info

## Daten vorbereiten

Die Grauwerte müssen wir zunächst von [0 ... 255] zu [0.0 ... 1.0] konvertieren.

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train_scaled = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test_scaled = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

Als nächstes erzeugen wir unseren Datengenerator, über den das Training nachher iteriert:

In [ ]:
train = ds_train_scaled.shuffle(1024, reshuffle_each_iteration=True).batch(64).prefetch(tf.data.experimental.AUTOTUNE)

Ebenso erzeugen wir unseren Test-Generator, allerdings hier ohne Shufflen.

In [ ]:
test = ds_test_scaled.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

# Convolutional Neural Net

...

## Definition des neuronalen Netzes



In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), input_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(64, (2,2)),
  tf.keras.layers.MaxPool2D((2,2)),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax()
])
print(model.summary())


In [ ]:
samples = ds_train_scaled.as_numpy_iterator()
[sample, label] = next(samples)
sample.shape, label.shape

## Compilation des Modells

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Training des Modells

In [ ]:
model.fit(train, epochs=10);

Unter den Epochen zeigt Tensorflow die Anzahl der Iterationsschritte pro Epoche an. Da wir eine
Batch-Size von 64 festgelegt hatten, bekommen wir 938 Schritte pro Epoche: wir benötigen 938
Minibatch Schritte bei einer Batchsize von 64, um die 60.000 Fälle abzudecken.

## Evaluation des Modells

Mit `Model.evaluate` überprüfen wir jetzt die Performanz des Modells mit unserem Test-Set:

In [ ]:
(loss, accuracy) = model.evaluate(test, verbose=1)
print(f'\nLoss: {loss:.4f}, Accuracy: {accuracy*100:.1f}%')